In [2]:
from main import *
import tensorflow as tf

from tqdm import tnrange, tqdm_notebook, tqdm

In [8]:
num_classes = 2
image_shape = (160, 576)
epochs = 15
batch_size = 5
data_dir = './data'
runs_dir = './runs'
tests.test_for_kitti_dataset(data_dir)

# Download pretrained vgg model
helper.maybe_download_pretrained_vgg(data_dir)

# Path to vgg model
vgg_path = os.path.join(data_dir, 'vgg')

Tests Passed


In [4]:
sess = tf.Session()

In [5]:
# OPTIONAL: Augment Images for better results
#  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

# TODO: Build NN using load_vgg, layers, and optimize function
input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
nn_layers = layers(layer3_out, layer4_out, layer7_out, num_classes)
learning_rate = tf.placeholder(tf.float32, name='learning_rate')
correct_label = tf.placeholder(tf.float32, shape=(None, None, None, num_classes), name='correct_label')
logits, train_op, cross_entropy_loss = optimize(nn_layers, correct_label, learning_rate, num_classes)

# Initialize TF variables
sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'


In [6]:
weights_path = './data/ckpt/model_25epoch_0071_good.ckpt'
saver = tf.train.Saver()

saver.restore(sess, weights_path)

INFO:tensorflow:Restoring parameters from ./data/ckpt/model_25epoch_0071_good.ckpt


In [9]:
get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

In [10]:
for ep in tnrange(epochs, unit=' epochs'):
    pbar = tqdm_notebook(enumerate(get_batches_fn(batch_size)), unit=' batches')
    for batch_idx, (image, label) in pbar:
        feed_dict = {input_image: image,
                     correct_label: label,
                     keep_prob: 0.50,
                     learning_rate: 1e-5,
                     }
        _, loss = sess.run([train_op, cross_entropy_loss], feed_dict)
        if batch_idx % 10 == 0:
            pbar.set_description('Epoch: %d, Batch: %d, Loss = %.3f' % (ep+1, batch_idx, loss))

In [11]:
helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

0 files [00:00, ? files/s]

Training Finished. Saving test images to: ./runs/1504595878.0752022


290 files [03:10,  1.53 files/s]


In [12]:
import time
saver = tf.train.Saver()

tf.add_to_collection('train_vars', input_image)
tf.add_to_collection('train_vars', correct_label)
tf.add_to_collection('train_vars', keep_prob)
tf.add_to_collection('train_vars', learning_rate)
tf.add_to_collection('train_vars', train_op)
tf.add_to_collection('train_vars', cross_entropy_loss)

tf.add_to_collection('eval_vars', input_image)
tf.add_to_collection('eval_vars', logits)
tf.add_to_collection('eval_vars', keep_prob)

tf.train.write_graph(sess.graph_def, 'data/ckpt', 'model.pb', as_text=False) # Save model
save_path = saver.save(sess, "./data/ckpt/model_40epoch_0050_final.ckpt") # Save weights

print("Model saved in file: %s" % save_path)

Model saved in file: ./data/ckpt/model_40epoch_0050_final.ckpt


In [13]:
sess.close()

In [ ]:
tf.reset_default_graph()
model_path = './data/ckpt/model.pb'
weights_path = './data/ckpt/model_20epoch_0119.ckpt'

train_elements = ['image_input:0', 'correct_label:0', 'keep_prob:0', 'learning_rate:0', 'train_op:0', 'loss:0']
eval_elements = ['image_input:0', 'logits:0', 'keep_prob:0']

with tf.Session() as sess:
    input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
    nn_layers = layers(layer3_out, layer4_out, layer7_out, num_classes)
    learning_rate = tf.placeholder(tf.float32)
    correct_label = tf.placeholder(tf.float32, shape=(None, None, None, num_classes))
    logits, train_op, cross_entropy_loss = optimize(nn_layers, correct_label, learning_rate, num_classes)

    saver = tf.train.Saver()
    saver.restore(sess, weights_path)
    helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)    

In [ ]:
def image_segmentation(image, sess, logits, keep_prob, image_pl, image_shape):
    """
    Generate mask for given image
    :param image: Image to be processed
    :param sess: TF session
    :param logits: TF Tensor for the logits
    :param keep_prob: TF Placeholder for the dropout keep robability
    :param image_pl: TF Placeholder for the image placeholder
    :param image_shape: Tuple - Shape of image
    :return: Output for for each test image
    """
    image = scipy.misc.imresize(image, image_shape)

    im_softmax = sess.run(
        [tf.nn.softmax(logits)],
        {keep_prob: 1.0, image_pl: [image]})
    im_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
    segmentation = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
    mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
    return mask

In [ ]:
# def load_graph(sess, graph_file):
#     gd = tf.GraphDef()
#     with tf.gfile.Open(graph_file, 'rb') as f:
#         data = f.read()
#         gd.ParseFromString(data)
#     tf.import_graph_def(gd, name='')
#     ops = sess.graph.get_operations()
#     n_ops = len(ops)
#     return sess.graph, ops

# use_xla = True
# jit_level = 0
# config = tf.ConfigProto()
# if use_xla:
#     jit_level = tf.OptimizerOptions.ON_1
#     config.graph_options.optimizer_options.global_jit_level = jit_level

# with tf.Session(config=config, graph=tf.Graph()) as sess:
#     graph, eightbit_ops = load_graph(sess, './data/ckpt/optimized_graph.pb')
#     graph.as_default()
#     sess.run(tf.global_variables_initializer())
#     logits = graph.get_tensor_by_name('logits:0')
#     keep_prob = graph.get_tensor_by_name('keep_prob:0')
#     input_image = graph.get_tensor_by_name('image_input:0')
#     helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

In [ ]:
# ~/tensorflow/bazel-bin/tensorflow/python/tools/freeze_graph \
# --input_graph=model.pb \
# --input_checkpoint=model_25epoch_0071_good.ckpt \
# --input_binary=true \
# --output_graph=frozen_graph.pb --output_node_names=logits


# ~/tensorflow/bazel-bin/tensorflow/python/tools/optimize_for_inference \
# --input=frozen_graph.pb \
# --output=optimized_graph.pb \
# --frozen_graph=True \
# --input_names=image_input \
# --output_names=logits

# # sess, optimized_ops = load_graph('optimized_graph.pb')
# # print(len(optimized_ops)) # 200

# ~/tensorflow/bazel-bin/tensorflow/tools/graph_transforms/transform_graph \
# --in_graph=frozen_graph.pb \
# --out_graph=eightbit_graph.pb \
# --inputs=image_input \
# --outputs=logits \
# --transforms='
# add_default_attributes
# remove_nodes(op=Identity, op=CheckNumerics)
# fold_constants(ignore_errors=true)
# fold_batch_norms
# fold_old_batch_norms
# fuse_resize_and_conv
# quantize_weights
# quantize_nodes
# strip_unused_nodes
# sort_by_execution_order'

# # sess, eightbit_ops = load_graph('eightbit_graph.pb')
# # print(len(optimized_ops)) # 425
